In [1]:
# Why are high salary earning employees leaving prematurely?


In [3]:
%fs ls /FileStore/tables/hrdata.csv

In [4]:
%sql DROP TABLE IF EXISTS adult

In [5]:
%sql

CREATE TABLE hrdata (
  satisfaction_level DOUBLE,
  last_evaluation DOUBLE,
  number_project INT,
  average_montly_hours INT,
  time_spend_company INT,
  Work_accident INT,
  left_company INT,
  promotion_last_5years INT,
  sales STRING,
  salary STRING)
USING com.databricks.spark.csv
OPTIONS (path "dbfs:/FileStore/tables/hrdata.csv", header "true")

In [6]:
%sql select * from hrdata

In [7]:
dataset = spark.table("hrdata")
cols = dataset.columns

In [8]:
display(dataset)

In [9]:
###One-Hot Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ["sales","salary"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]

In [10]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "left_company", outputCol = "label")
stages += [label_stringIdx]

In [11]:
# Transform all features into a vector using VectorAssembler
numericCols = ["satisfaction_level", "last_evaluation", "number_project", "average_montly_hours", "time_spend_company", "Work_accident","promotion_last_5years"]
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [12]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(dataset)
dataset = pipelineModel.transform(dataset)

# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)
display(dataset)

In [13]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print trainingData.count()
print testData.count()

In [14]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [15]:
predictions = lrModel.transform(testData)

In [16]:
predictions.printSchema()

In [17]:
selected = predictions.select("label", "prediction", "probability", "salary", "sales")
display(selected)

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

In [19]:
evaluator.getMetricName()

In [20]:
print lr.explainParams()

In [21]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [22]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [23]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [24]:
evaluator.evaluate(predictions)

In [25]:
print 'Model Intercept: ', cvModel.bestModel.intercept

In [26]:
# View best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "salary","sales")
display(selected)

In [27]:
#Decision Trees

In [28]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)

# Train model with Training Data
dtModel = dt.fit(trainingData)

In [29]:
print "numNodes = ", dtModel.numNodes
print "depth = ", dtModel.depth

In [30]:
# Make predictions on test data using the Transformer.transform() method.
predictions = dtModel.transform(testData)

In [31]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "salary", "sales")
display(selected)

In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

In [33]:
dt.setImpurity("Entropy")
dt.getImpurity()

In [34]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1,2,6,10])
             .addGrid(dt.maxBins, [20,40,80])
             .build())

In [35]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# Takes ~5 minutes

In [36]:
print "numNodes = ", cvModel.bestModel.numNodes
print "depth = ", cvModel.bestModel.depth

In [37]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [38]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

In [39]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "salary", "sales")
display(selected)